In [1]:
%cd cache-prefetchers
#! git pull

!pip install sympy

# make a callback that does !git pull before running each cell
from IPython import get_ipython

def git_pull():
    !git pull > /dev/null

get_ipython().events.register('pre_run_cell', git_pull)



/notebooks/cache-prefetchers


In [2]:
%load_ext autoreload
%autoreload 3


In [6]:
import matplotlib.pyplot as plt
import numpy as np
from typing import List, Tuple, Set
import abc
import dataclasses
import pandas as pd
import os
import copy
import time
import tqdm


from cache import Cache
from eviction import LRU, DOM, MQ, RandomEvictor
from prefetch import Next, Markov, EnsamblePrefetcher
from prob_model import FFM
from markov_generator import RandomMarkovGenerator
from fieldfm import train_ffm, FieldAwareFactorizationMachine, FieldAwareFactorizationMachineModel


SyntaxError: EOL while scanning string literal (fieldfm.py, line 26)

In [12]:


# Those are somewhat good parameters
n = 1000  # Number of distinct pages
h = 14  # History length
m = h+2    # Number of fields (example value, h + 2)
k = 10   # Size of the latent factors

MY_FFM = False
if MY_FFM:
    ffm = FieldAwareFactorizationMachine(n, m, k)
else:
    ffm = FieldAwareFactorizationMachineModel([n]*m, k)

num_parameters = sum(p.numel() for p in ffm.parameters() if p.requires_grad)
print(f'Liczba parametrów: {num_parameters}')

history = RandomMarkovGenerator(n).generate_sequence(30000)

losses = []
for i in range(2000):
    cache = set(np.random.randint(0, n, size=20))
    losses += train_ffm(ffm, history, cache, h, epochs=1, lr=0.01, wd=0.1, epoch_samples=4)

context = history[:h]
a, b = np.random.choice(list(cache), 2, replace=False)

print(f"p({a}, {b}) = ", ffm([a, b]+ context))
print(f"p({b}, {a}) = ", ffm([b, a]+ context))
print(f"sum = ", ffm([a, b]+ context) + ffm([b, a]+ context))

plt.plot(losses)
print("losses", losses[-5:])


Liczba parametrów: 2576001


TypeError: 'set' object is not subscriptable

In [4]:
import torch

# is cuda available?
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


# Tests on Markov generated traces

In [3]:
SEQ_LEN = 50000
N = 500  # Address space size
SEQ_COUNT = 3  # Number of sequences to generate
CACHE_SIZE = 8

sequences = [
    RandomMarkovGenerator(N).generate_sequence(SEQ_LEN)
    for _ in range(SEQ_COUNT)
]


@dataclasses.dataclass
class TestResult:

    cache_name: str
    miss_count: int
    trace_length: int
    unique_addresses: int
    execution_time: float


    def __repr__(self) -> str:
        return (
            f"Cache name: {self.cache_name}\n"
            f"Miss count: {self.miss_count}\n"
            f"Trace length: {self.trace_length}\n"
            f"Unique addresses: {self.unique_addresses}\n"
            f"Miss rate: {self.miss_count / self.trace_length}\n"
            f"Execution time: {self.execution_time}"
        )



def test_cache(cache: Cache, requests: List[int]) -> TestResult:
    """Returns cache miss count for given cache and requests."""
    cache = copy.deepcopy(cache)
    time_start = time.time()

    misses = 0
    for r in tqdm.tqdm(requests):
        if not cache.read(r):
            misses += 1

    time_end = time.time()

    return TestResult(
        cache_name=cache.name,
        miss_count=misses,
        trace_length=len(requests),
        unique_addresses=len(set(requests)),
        execution_time=time_end - time_start,
    )


NameError: name 'RandomMarkovGenerator' is not defined

In [8]:
def test_markov_sequences(caches: List[Cache], sequences: List[List[int]], cache_size: int):
    file_name = f"markov_sequences_{cache_size}.csv"

    if os.path.exists(file_name):
        res = pd.read_csv(file_name, index_col=0)
    else:
        res = pd.DataFrame()

    for i, seq in enumerate(sequences):
        for cache in caches:

            if len(res) > 0 and len(res[(res['cache'] == cache.name) & (res['seq'] == i)]) > 0:
                continue

            print("=======================================")
            print(f"Sequence {i}, cache: {cache.name}")
            print("=======================================")
            try:
                test_result = test_cache(cache, seq)
            except Exception as exc:
                print("ERROR", exc)
                continue

            new_row = pd.DataFrame([{
                'seq': i,
                'cache': cache.name,
                'miss_count': test_result.miss_count,
                'trace_length': test_result.trace_length,
                'unique_addresses': test_result.unique_addresses,
                'execution_time': test_result.execution_time
            }])
            res = pd.concat([res, new_row])
            res.to_csv(file_name)

    res['miss_rate'] = res['miss_count'] / res['trace_length']
    res = res.sort_values(by=['seq', 'miss_rate'])

    return res


In [12]:
caches = [
    Cache(
        name="RandomEvictor",
        eviction_strategy=RandomEvictor(),
        size=CACHE_SIZE
    ),
    Cache(
        name="LRU",
        eviction_strategy=LRU(),
        size=CACHE_SIZE
    ),
    Cache(
        name="MQ",
        eviction_strategy=MQ(),
        size=CACHE_SIZE
    ),
    Cache(
        name="DOM(FFM(n=N, h=1, k=10))",
        eviction_strategy=DOM(FFM(n=N, h=1, k=10, my_ffm=False), train_interval=50),
        size=CACHE_SIZE
    ),
    Cache(
        name="RandomEvictor + NEXT",
        eviction_strategy=RandomEvictor(),
        prefetch_strategy=Next(),
        size=CACHE_SIZE
    ),
    Cache(
        name="LRU + NEXT",
        eviction_strategy=LRU(),
        prefetch_strategy=Next(),
        size=CACHE_SIZE
    ),
    Cache(
        name="MQ + NEXT",
        eviction_strategy=MQ(),
        prefetch_strategy=Next(),
        size=CACHE_SIZE
    ),
    Cache(
        name="DOM(FFM(n=N, h=14, k=10)) + NEXT",
        eviction_strategy=DOM(FFM(n=N, h=14, k=10), train_interval=50),
        prefetch_strategy=Next(),
        size=CACHE_SIZE
    ),
    Cache(
        name="RandomEvictor + MARKOV(3)",
        eviction_strategy=RandomEvictor(),
        prefetch_strategy=Markov(3),
        size=CACHE_SIZE
    ),
    Cache(
        name="LRU + MARKOV(3)",
        eviction_strategy=LRU(),
        prefetch_strategy=Markov(3),
        size=CACHE_SIZE
    ),
    Cache(
        name="MQ + MARKOV(3)",
        eviction_strategy=MQ(),
        prefetch_strategy=Markov(3),
        size=CACHE_SIZE
    ),
    Cache(
        name="DOM(FFM(n=N, h=14, k=10)) + MARKOV(3)",
        eviction_strategy=DOM(FFM(n=N, h=14, k=10)),
        prefetch_strategy=Markov(3),
        size=CACHE_SIZE
    ),
][:3]



for h in [2]:
    for k in [10]:
        for train_interval in [10, 50]:
            for samples in [200, 50]:
                for epochs in [4, 2]:
                    caches.append(
                        Cache(
                            name=f"DOM(FFM(n=N, h={h}, k={k} e_s={samples} t_i={train_interval} e={epochs}))",
                            eviction_strategy=DOM(FFM(n=N, h=h, k=k, epoch_samples=samples), train_interval=train_interval),
                            size=CACHE_SIZE
                        )
                    )

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
test_markov_sequences(caches, sequences, CACHE_SIZE)

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0
Unpacking objects: 100% (3/3), 779 bytes | 86.00 KiB/s, done.
From https://github.com/rllaskowski/cache-prefetchers
   10ac488..c06ab7a  main       -> origin/main
Sequence 0, cache: DOM(FFM(n=N, h=2, k=10 e_s=200 t_i=10 e=4))


  0%|          | 8/50000 [00:00<00:03, 16131.94it/s]


ERROR 'numpy.ndarray' object has no attribute 'device'
Sequence 0, cache: DOM(FFM(n=N, h=2, k=10 e_s=200 t_i=10 e=2))


  0%|          | 8/50000 [00:00<00:03, 16619.33it/s]


ERROR 'numpy.ndarray' object has no attribute 'device'
Sequence 0, cache: DOM(FFM(n=N, h=2, k=10 e_s=50 t_i=10 e=4))


...

  0%|          | 8/50000 [00:00<00:03, 12609.71it/s]


ERROR 'numpy.ndarray' object has no attribute 'device'
Sequence 2, cache: DOM(FFM(n=N, h=2, k=10 e_s=50 t_i=50 e=2))


  0%|          | 8/50000 [00:00<00:05, 9310.33it/s]

ERROR 'numpy.ndarray' object has no attribute 'device'


,seq,cache,miss_count,trace_length,unique_addresses,execution_time,miss_rate
0,0,RandomEvictor,49173,50000,500,1.156051,0.98346
0,0,LRU,49194,50000,500,0.662641,0.98388
0,0,MQ,49196,50000,500,2.394601,0.98392
0,1,MQ,49152,50000,500,2.295167,0.98304
0,1,LRU,49198,50000,500,0.693560,0.98396
0,1,RandomEvictor,49201,50000,500,1.264985,0.98402
0,2,LRU,49183,50000,500,0.763505,0.98366
0,2,MQ,49204,50000,500,2.635250,0.98408
0,2,RandomEvictor,49216,50000,500,1.253592,0.98432


,seq,cache,miss_count,trace_length,unique_addresses,execution_time,miss_rate
0,0,"DOM(FFM(n=N, h=2, k=10 epoch_samples=50 train_interval=50))",49147,50000,500,142.775758,0.98294
0,0,"DOM(FFM(n=N, h=1, k=10 epoch_samples=50 train_interval=50))",49149,50000,500,142.684629,0.98298
0,0,LRU,49159,50000,500,0.401157,0.98318
0,0,"DOM(FFM(n=N, h=1, k=5 epoch_samples=20 train_interval=50))",49161,50000,500,70.570963,0.98322
0,0,"DOM(FFM(n=N, h=2, k=10 epoch_samples=20 train_interval=50))",49166,50000,500,80.666794,0.98332
0,0,"DOM(FFM(n=N, h=4, k=5 epoch_samples=50 train_interval=100))",49174,50000,500,95.375585,0.98348
0,0,"DOM(FFM(n=N, h=4, k=5 epoch_samples=50 train_interval=50))",49178,50000,500,148.745023,0.98356
0,0,"DOM(FFM(n=N, h=2, k=10 epoch_samples=20 train_interval=100))",49179,50000,500,58.644135,0.98358
0,0,"DOM(FFM(n=N, h=4, k=10 epoch_samples=20 train_interval=100))",49179,50000,500,65.313656,0.98358
0,0,"DOM(FFM(n=N, h=1, k=10 epoch_samples=20 train_interval=100))",49180,50000,500,58.054378,0.98360


Sequence 0, cache: RandomEvictor


100%|██████████| 50000/50000 [00:00<00:00, 86393.82it/s] 


Sequence 0, cache: LRU


100%|██████████| 50000/50000 [00:00<00:00, 149584.16it/s]


Sequence 0, cache: MQ


...

Sequence 2, cache: MQ


100%|██████████| 50000/50000 [00:01<00:00, 42205.10it/s]


Sequence 2, cache: DOM(FFM(n=N, h=14, k=10))


100%|██████████| 50000/50000 [02:35<00:00, 321.40it/s]


,seq,cache,miss_count,trace_length,unique_addresses,execution_time,miss_rate
0,0,LRU,49159,50000,500,0.335613,0.98318
0,0,RandomEvictor,49178,50000,500,0.597701,0.98356
0,0,"DOM(FFM(n=N, h=14, k=10))",49183,50000,500,139.674669,0.98366
0,0,MQ,49214,50000,500,1.210652,0.98428
0,1,MQ,49190,50000,500,1.193911,0.98380
0,1,LRU,49200,50000,500,0.344964,0.98400
0,1,"DOM(FFM(n=N, h=14, k=10))",49209,50000,500,154.673867,0.98418
0,1,RandomEvictor,49215,50000,500,0.619806,0.98430
0,2,RandomEvictor,49194,50000,500,0.570563,0.98388
0,2,MQ,49204,50000,500,1.185715,0.98408


In [64]:
# To check if it's learning anything
plt.plot(cache.eviction_strategy.prob_model.losses)
plt.title("Losses of the FFM model")
plt.show()


AttributeError: 'set' object has no attribute 'eviction_strategy'

# Tests on HUAWEI traces

In [7]:
from hwi_dataset import read_trace, get_test_set, get_reads, get_page_requests

test_set = get_test_set()

read_trace(test_set['1'][0][0])


,timestamp,tp,opCode,result,volumeId,objId,objLba,length,timepoint,last time nano,print data
NaN,Thu Nov 2 02:04:58 2017,READ_DONE,0x128001,0,0x251,0x251,258106256,16,1,18648278912419089,5a93dc390000a206
NaN,Thu Nov 2 02:04:58 2017,READ,0x128001,0,0x251,0x251,57317984,16,1,18648278913355764,0
NaN,Thu Nov 2 02:04:58 2017,PREFETCH,0x128001,0,0x251,0x251,57317984,16,1,18648278913376335,0
NaN,Thu Nov 2 02:04:58 2017,PREFETCH_DONE,0x128001,0,0x251,0x251,57317984,16,1,18648278913742002,139bc7a60000a206
NaN,Thu Nov 2 02:04:58 2017,READ_DONE,0x128001,0,0x251,0x251,57317984,16,1,18648278913748938,139bc7a60000a206
...,...,...,...,...,...,...,...,...,...,...,...
NaN,Thu Nov 2 02:06:50 2017,WRITE_DONE,0x13218000,0,0x251,0x251,498558608,16,1,18648390221699964,928bb3690000a206
NaN,Thu Nov 2 02:06:50 2017,WRITE,0x13218000,0,0x251,0x251,515518656,16,1,18648390236760913,95ca4c0c0000a206
NaN,Thu Nov 2 02:06:50 2017,VOL_SUBREQ,0x13218000,0,0x251,0x251,515518656,16,1,18648390236779367,95ca4c0c0000a206
NaN,Thu Nov 2 02:06:50 2017,WRITE_MIR,0x13218000,0,0x251,0x251,515518656,16,1,18648390236799367,95ca4c0c0000a206


In [8]:
import dataclasses
import copy
import tqdm
import math


def test_case(test_dir_name: str, cache: Cache, page_size=16*1024) -> TestResult:
    start_time = time.time()
    total_misses = 0
    total_length = 0

    unique_addresses = set()
    #uc = set()
    cache_factory = lambda: copy.deepcopy(cache)

    for volume_paths in sorted(test_set[test_dir_name]):
        cache = cache_factory()

        print(f"Volume {volume_paths[0].split('/')[-2]}")

        for path in tqdm.tqdm(volume_paths):
            hits = 0

            requests = get_page_requests(get_reads(read_trace(path)), page_size=page_size)

            for r in requests:
                unique_addresses.add(r)
                #uc.add((i, r))
                hit = cache.read(r)
                hits += hit

            total_misses += len(requests) - hits
            total_length += len(requests)

    return TestResult(
        cache_name=cache.name,
        miss_count=total_misses,
        unique_addresses=len(unique_addresses),
        trace_length=total_length,
        execution_time=round(time.time() - start_time, 3)
    )


In [9]:
from opt import opt_cache_simulation


def get_opt_for_test_dir(test_set_dir, page_size=16*1024, cache_size=8) -> TestResult:
    total_misses = 0
    total_length = 0
    unique_addresses = set()

    for volume_paths in sorted(test_set[test_set_dir]):

        for path in volume_paths:
            requests = get_page_requests(get_reads(read_trace(path)), page_size=page_size)
            total_length += len(requests)
            unique_addresses.update(requests)

            misses = opt_cache_simulation(volume_paths, cache_size)
            total_misses += misses

    return TestResult(
        cache_name="OPT",
        miss_count=total_misses,
        unique_addresses=len(unique_addresses),
        trace_length=total_length,
        execution_time=None
    )


In [10]:
import pandas as pd
import os

test_dirs = [
    '1',
    '2',
    '3',
    '11',
    'scenario_test_trace_simple/VDI_virus_scan'
]

def test(caches: List[Cache], test_dirs: List[str], cache_size, page_size=16*1024) -> pd.DataFrame:
    file_name = f'results_{page_size}_{cache_size}.csv'
    if os.path.exists(file_name):
        res = pd.read_csv(file_name, dtype={'test_dir': str}, index_col=0)
    else:
        res = pd.DataFrame()

    for test_dir in test_dirs:
        for cache in caches:

            if len(res) > 0 and len(res[(res['cache'] == cache.name) & (res['test_dir'] == test_dir)]) > 0:
                continue


            print("=======================================")
            print(f"Test dir: {test_dir}, cache: {cache.name}")
            print("=======================================")
            try:
                test_result = test_case(test_dir, cache, page_size=page_size)
            except Exception as exc:
                print("ERROR", exc)
                continue

            new_row = pd.DataFrame([{
                'test_dir': test_dir,
                'cache': cache.name,
                'miss_count': test_result.miss_count,
                'trace_length': test_result.trace_length,
                'unique_addresses': test_result.unique_addresses,
                'page_size': page_size,
                'execution_time': test_result.execution_time
            }])
            res = pd.concat([res, new_row])
            res.to_csv(file_name)

        if len(res) > 0 and len(res[(res['cache'] == 'OPT') & (res['test_dir'] == test_dir)]) > 0:
            continue

        opt_res = get_opt_for_test_dir(test_dir, page_size=page_size, cache_size=cache_size)
        new_row = pd.DataFrame([{
            'test_dir': test_dir,
            'cache': 'OPT',
            'miss_count': opt_res.miss_count,
            'trace_length': opt_res.trace_length,
            'unique_addresses': opt_res.unique_addresses,
            'page_size': page_size,
            'execution_time': opt_res.execution_time
        }])
        res = pd.concat([res, new_row])
        res.to_csv(file_name)

    res['miss_rate'] = res['miss_count'] / res['trace_length']
    res = res.sort_values(by=['test_dir', 'miss_rate'])

    return res


In [125]:
from os import replace
from tkinter import SE

from torch import cudnn_convolution_add_relu
from eviction import solve_linear_program
import random
from pprint import pprint
import functools
import itertools


def test_dom():
    ss = []
    ds = []
    for i in range(1000):
        N = 5
        pages = list(range(N))
        probs = {}
        for i in pages:
            for j in pages:
                if i == j:
                    continue
                probs[(i, j)] = random.random()
                probs[(j, i)] = 1 - probs[(i, j)]

        distr = solve_linear_program(probs, pages)
        assert distr is not None
        #pprint(probs)
        s = {}
        for i in pages:
            s[i] = sum(probs[(i, j)] for j in pages if j != i)
            #print(i, ':', s)
        ss.append(s[0])
        ds.append(distr[0])
        #print(distr)

    # calculate correlation
    print(np.corrcoef(ss, ds))

#test_dom()



def test_dom2():
    N = 50
    SEQ_LEN = 1000_00
    SAMPLES = 50000
    pages = range(N)

    probs = np.random.uniform(size=[N])
    probs = probs / sum(probs)

    sequence = [random.choices(pages, k=1, weights=probs)[0] for _ in range(SEQ_LEN)]
    not_before_occ = {(i,j): 0 for i, j in itertools.permutations(pages, 2)}

    for i in tqdm.tqdm(range(50000)):
        t = random.randint(0, SEQ_LEN)
        a, b = random.sample(pages, k=2)
        if a == b:
            continue

        for x in sequence[t:]:
            if x == a:
                not_before_occ[(b,a)] += 1

    prob_c = {}
    for a in pages:
        for b in pages:
            if a == b:
                continue
            s = not_before_occ[a,b] + not_before_occ[b,a]

            if s == 0:
                prob_c[(a,b)] = 0.5
            else:
                prob_c[(a,b)] = not_before_occ[(a,b)] / s

    cache_size = 8


    hits_dom = 0
    hits_random = 0
    better = 0
    worse = 0
    for i in tqdm.tqdm(range(SAMPLES)):
        cache = random.sample(pages, k=cache_size)

        probs_dom = solve_linear_program(prob_c, cache)
        t = random.randint(0, len(sequence))

        w = random.choices(cache, k=1, weights=probs_dom)[0]
        occ = set()
        a_ = 0
        for a in sequence[t:]:
            if a in cache and a != w:
                occ.add(a)
            if a == w:
                if len(occ) == len(cache) -1:
                    hits_dom += 1
                    a_ = 1
                break

        w = random.choices(cache, k=1)[0]
        occ = set()
        b = 0
        for a in sequence[t:]:
            if a in cache and a != w:
                occ.add(a)
            if a == w:
                if len(occ) == len(cache) -1:
                    hits_random += 1
                    b = 1
                break

        if a_ > b:
            better += 1

        if b > a_:
            worse += 1

    print("hits_dom", hits_dom, hits_dom/SAMPLES)
    print("hits_random", hits_random, hits_random/SAMPLES)
    print("better", better, better/SAMPLES)
    print("worse", worse, worse/SAMPLES)


test_dom2()

100%|██████████| 50000/50000 [00:52<00:00, 959.65it/s]

hits_dom 20546 0.41092
hits_random 6394 0.12788
better 17964 0.35928
worse 3812 0.07624


In [126]:
caches_size_8 = [
    Cache(
        "RandomEvictor",
        eviction_strategy=RandomEvictor(),
        size=8
    ),
    Cache(
        "LRU",
        eviction_strategy=LRU(),
        size=8
    ),
    Cache(
        "MQ",
        eviction_strategy=MQ(max_frequency=8, life_time=10, qout_size=8*4),
        size=8
    ),
    Cache(
        "DOM(FFM(n=10000,k=10,h=14))",
        eviction_strategy=DOM(prob_model=FFM(n=10009, k=10, h=14, epoch_samples=10), train_interval=50),
        size=8
    ),
    Cache(
        "LRU + NEXT",
        eviction_strategy=LRU(),
        prefetch_strategy=Next(),
        size=8
    ),
    Cache(
        "LRU + NEXT",
        eviction_strategy=LRU(),
        prefetch_strategy=Next(),
        size=8
    ),
    Cache(
        "LRU + Markov(3)",
        eviction_strategy=LRU(),
        prefetch_strategy=Markov(3),
        size=8
    ),
    Cache(
        "MQ + NEXT",
        eviction_strategy=MQ(max_frequency=8, life_time=10, qout_size=8*4),
        prefetch_strategy=Next(),
        size=8
    ),
    Cache(
        "MQ + Markov(3)",
        eviction_strategy=MQ(max_frequency=8, life_time=10, qout_size=8*4),
        prefetch_strategy=Markov(3),
        size=8
    ),
    Cache(
        "DOM(FFM(n=10000,k=10,h=14)) + NEXT",
        eviction_strategy=DOM(prob_model=FFM(n=1000, k=10, h=14)),
        prefetch_strategy=Next(),
        size=8
    ),
    Cache(
        "DOM(FFM(n=10000,k=10,h=14)) + Ensamble(NEXT+Markov(3))",
        eviction_strategy=DOM(prob_model=FFM(n=1000, k=10, h=14)),
        prefetch_strategy=EnsamblePrefetcher([Next(),Markov(3)]),
        p=2,
        size=8
    ),
][3:4]

test(caches_size_8, test_dirs, 8, page_size=16*1024)


Test dir: 1, cache: DOM(FFM(n=10000,k=10,h=14))
Volume cache_trace_593


100%|██████████| 31/31 [03:43<00:00,  7.21s/it]
/var/folders/wv/wmn__gfj1j188w6fb4gdzvn80000gn/T/ipykernel_53635/3219294522.py:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  res = pd.concat([res, new_row])


Test dir: 2, cache: DOM(FFM(n=10000,k=10,h=14))
Volume cache_trace_3


100%|██████████| 11/11 [00:12<00:00,  1.18s/it]
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x1037377c0>>
Traceback (most recent call last):
  File "/Users/robertlaskowski/Desktop/studia/.venv/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


Volume cache_trace_6


 29%|██▉       | 5/17 [00:11<00:22,  1.90s/it]

In [525]:

caches_size_16 = [
    Cache(
        "RandomEvictor",
        eviction_strategy=RandomEvictor(),
        size=16
    ),
    Cache(
        "LRU",
        eviction_strategy=LRU(),
        size=16
    ),
    Cache(
        "MQ",
        eviction_strategy=MQ(max_frequency=8, life_time=10, qout_size=8*4),
        size=16
    ),
    Cache(
        "DOM(FFM(n=10000,k=10,h=14))",
        eviction_strategy=DOM(prob_model=FFM(n=10000, k=10, h=14)),
        size=16
    ),
    Cache(
        "LRU + Markov(3)",
        eviction_strategy=LRU(),
        prefetch_strategy=Markov(3),
        size=16
    ),
    Cache(
        "LRU + NEXT",
        eviction_strategy=LRU(),
        prefetch_strategy=Next(),
        size=16
    ),
    Cache(
        "MQ + NEXT",
        eviction_strategy=MQ(max_frequency=8, life_time=10, qout_size=8*4),
        prefetch_strategy=Next(),
        size=16
    ),
    Cache(
        "MQ + Markov(3)",
        eviction_strategy=MQ(max_frequency=8, life_time=10, qout_size=8*4),
        prefetch_strategy=Markov(3),
        size=16
    ),
    Cache(
        "DOM(FFM(n=10000,k=10,h=14)) + NEXT",
        eviction_strategy=DOM(prob_model=FFM(n=1000, k=10, h=14)),
        prefetch_strategy=Next(),
        size=16
    ),
]


In [531]:
test(caches_size_16, test_dirs, 16, page_size=16*1024)


,test_dir,cache,miss_count,trace_length,unique_addresses,page_size,execution_time,miss_rate
0,1,OPT,961,61932,14281,16384,NaN,0.015517
0,1,MQ,50465,61932,14281,16384,6.924,0.814845
0,1,LRU,50600,61932,14281,16384,3.742,0.817025
0,1,MQ + Markov(3),51013,61932,14281,16384,5.817,0.823694
0,1,MQ + NEXT,51043,61932,14281,16384,5.701,0.824178
0,1,RandomEvictor,51248,61932,14281,16384,3.988,0.827488
0,1,LRU + Markov(3),51446,61932,14281,16384,4.239,0.830685
0,1,LRU + NEXT,51477,61932,14281,16384,4.532,0.831186
0,1,"DOM(FFM(n=10000,k=10,h=14))",52188,61932,14281,16384,620.904,0.842666
0,1,"DOM(FFM(n=10000,k=10,h=14)) + NEXT",55543,61932,14281,16384,1000.834,0.896838


In [533]:

caches_size_64 = [
    Cache(
        "RandomEvictor",
        eviction_strategy=RandomEvictor(),
        size=64
    ),
    Cache(
        "LRU",
        eviction_strategy=LRU(),
        size=64
    ),
    Cache(
        "MQ",
        eviction_strategy=MQ(max_frequency=8, life_time=10, qout_size=8*4),
        size=64
    ),
    Cache(
        "LRU + NEXT",
        eviction_strategy=LRU(),
        prefetch_strategy=Next(),
        size=64
    ),
    Cache(
        "LRU + NEXT",
        eviction_strategy=LRU(),
        prefetch_strategy=Next(),
        size=64
    ),
    Cache(
        "LRU + Markov(3)",
        eviction_strategy=LRU(),
        prefetch_strategy=Markov(3),
        size=16
    ),
    Cache(
        "MQ + NEXT",
        eviction_strategy=MQ(max_frequency=8, life_time=10, qout_size=8*4),
        prefetch_strategy=Next(),
        size=64
    ),
    Cache(
        "MQ + Markov(3)",
        eviction_strategy=MQ(max_frequency=8, life_time=10, qout_size=8*4),
        prefetch_strategy=Markov(3),
        size=64
    ),
]
test(caches_size_64, test_dirs, 64, page_size=16*1024)


,test_dir,cache,miss_count,trace_length,unique_addresses,page_size,execution_time,miss_rate
0,1,OPT,961,61932,14281,16384,None,0.015517
0,1,MQ,48394,61932,14281,16384,9.215,0.781405
0,1,LRU,48415,61932,14281,16384,3.677,0.781744
0,1,RandomEvictor,48952,61932,14281,16384,3.942,0.790415
0,1,MQ + Markov(3),49298,61932,14281,16384,8.863,0.796002
0,1,MQ + NEXT,49328,61932,14281,16384,9.126,0.796486
0,1,LRU + NEXT,49363,61932,14281,16384,4.099,0.797052
0,1,LRU + Markov(3),51446,61932,14281,16384,4.332,0.830685
0,1,"DOM(FFM(n=10000,k=10,h=6))",52467,61932,14281,16384,2194.868,0.847171
0,1,"DOM(FFM(n=10000,k=10,h=6)) + NEXT",57120,61932,14281,16384,4587.174,0.922302


## Tests on smaller page sizes

In [ ]:

caches_size_16_smaller_pages = [
    Cache(
        "RandomEvictor",
        eviction_strategy=RandomEvictor(),
        size=16
    ),
    Cache(
        "LRU",
        eviction_strategy=LRU(),
        size=16
    ),
    Cache(
        "MQ",
        eviction_strategy=MQ(max_frequency=8, life_time=10, qout_size=8*4),
        size=16
    ),
    Cache(
        "LRU + Markov(3)",
        eviction_strategy=LRU(),
        prefetch_strategy=Markov(3),
        size=16
    ),
    Cache(
        "LRU + NEXT",
        eviction_strategy=LRU(),
        prefetch_strategy=Next(),
        size=16
    ),
    Cache(
        "MQ + NEXT",
        eviction_strategy=MQ(max_frequency=8, life_time=10, qout_size=8*4),
        prefetch_strategy=Next(),
        size=16
    ),
    Cache(
        "MQ + Markov(3)",
        eviction_strategy=MQ(max_frequency=8, life_time=10, qout_size=8*4),
        prefetch_strategy=Markov(3),
        size=16
    ),
]


### Page size - 512 Bytes

In [532]:
test(caches_size_64, test_dirs, 16, page_size=512)


,test_dir,cache,miss_count,trace_length,unique_addresses,page_size,execution_time,miss_rate
0,1,OPT,961,61932,41935,512,None,0.015517
0,1,MQ + Markov(3),51928,61932,41935,512,14.455,0.838468
0,1,MQ + NEXT,51956,61932,41935,512,14.73,0.838920
0,1,LRU + NEXT,51961,61932,41935,512,6.371,0.839001
0,1,LRU + Markov(3),53040,61932,41935,512,7.199,0.856423
0,1,LRU,53757,61932,41935,512,9.004,0.868000
0,1,MQ,53765,61932,41935,512,10.213,0.868130
0,1,RandomEvictor,54045,61932,41935,512,6.805,0.872651
0,11,OPT,209,57867,29936,512,None,0.003612
0,11,MQ + Markov(3),45893,57867,29936,512,15.915,0.793077
